In [1]:
#Loading Libraries
import numpy as np 
import pandas as pd
import numpy as np
import os
import imutils
import dlib 
import cv2 
import imageio
from imutils import face_utils

In [2]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
    	coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

rect_to_bb: Function to convert a bounding box predicted by dlib to (x, y, w, h) format as used in OpenCV

rect: dlib rectangle objec

shape_to_np: Function to convert facial landmark coordinates from dlib's shape predictor to NumPy array format
shape: dlib shape object, representing facial landmarks# dtype: data type of the output NumPy array, default is 'int't



In [3]:
def crop_and_save_image(img, img_path, write_img_path, img_name):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('D:/Lipreading/MIRACL-VC1_all_in_one/shape_predictor_68_face_landmarks.dat')
    # load the input image, resize it, and convert it to grayscale

    image = cv2.imread(img_path)
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    if len(rects) > 1:
    	print( "ERROR: more than one face detected")
    	return
    if len(rects) < 1:
    	print( "ERROR: no faces detected")
    	return

    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        name, i, j = 'mouth', 48, 68
        clone = gray.copy()

        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))        
        roi = gray[y:y+h, x:x+w]
        roi = imutils.resize(roi, width = 250, inter=cv2.INTER_CUBIC)        
        print('cropped/' + write_img_path)
        imageio.imwrite('cropped/' + write_img_path, roi)


In [4]:
os.listdir('D:/Lipreading/MIRACL-VC1_all_in_one')
predictor = dlib.shape_predictor('D:/Lipreading/MIRACL-VC1_all_in_one/shape_predictor_68_face_landmarks.dat')

In [5]:

people = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
data_types = ['words']
folder_enum = ['01','02','03','04','05','06','07','08','09','10']
instances = ['01','02','03','04','05','06','07','08','09','10']

words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']          
words_di = {i:words[i] for i in range(len(words))}

In [6]:
#crearting new folder for cropped images
#if not os.path.exists('cropped'):
    #os.mkdir('cropped')

In [7]:
import shutil

def crop_one_person():      
    
#    people = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
#    data_types = ['words']
#   folder_enum = ['01','02']
#    instances = ['01','02']

    i = 1
    for person_ID in people:
        if not os.path.exists('cropped/' + person_ID ):
            os.mkdir('cropped/' + person_ID + '/')

        for data_type in data_types:
            if not os.path.exists('cropped/' + person_ID + '/' + data_type):
                os.mkdir('cropped/' + person_ID + '/' + data_type)

            for phrase_ID in folder_enum:
                if not os.path.exists('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID):
                    # F01/phrases/01
                    os.mkdir('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID)

                for instance_ID in instances:
                    # F01/phrases/01/01
                    directory = 'D:/Lipreading/MIRACL-VC1_all_in_one/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID + '/'
                    dir_temp = person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID + '/'
                    print(directory)
                    filelist = os.listdir(directory)
                    if not os.path.exists('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID):
                        os.mkdir('cropped/' + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID)

                        for img_name in filelist:
                            if img_name.startswith('color'):
                                image = imageio.imread(directory + '' + img_name)
                                crop_and_save_image(image, directory + '' + img_name,
                                                    dir_temp + '' + img_name, img_name)

    print(f'Iteration : {i}')
    i += 1
  #  shutil.rmtree('cropped')

In [42]:
import time
os.mkdir('cropped')
times = 0
for _ in range(7):
    t1 = time.time()
    crop_one_person()
    t2 = time.time()
    times += (t2 - t1)

print("Average time over 7 iterations : ", times/7)

D:/Lipreading/MIRACL-VC1_all_in_one/F01/words/01/01/


C:\Users\satvi\AppData\Local\Temp\ipykernel_22876\2549150130.py:35: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(directory + '' + img_name)


cropped/F01/words/01/01/color_001.jpg
cropped/F01/words/01/01/color_002.jpg
cropped/F01/words/01/01/color_003.jpg
cropped/F01/words/01/01/color_004.jpg
cropped/F01/words/01/01/color_005.jpg
cropped/F01/words/01/01/color_006.jpg
cropped/F01/words/01/01/color_007.jpg
cropped/F01/words/01/01/color_008.jpg
cropped/F01/words/01/01/color_009.jpg
cropped/F01/words/01/01/color_010.jpg
D:/Lipreading/MIRACL-VC1_all_in_one/F01/words/01/02/
cropped/F01/words/01/02/color_001.jpg
cropped/F01/words/01/02/color_002.jpg
cropped/F01/words/01/02/color_003.jpg
cropped/F01/words/01/02/color_004.jpg
cropped/F01/words/01/02/color_005.jpg
cropped/F01/words/01/02/color_006.jpg
cropped/F01/words/01/02/color_007.jpg
D:/Lipreading/MIRACL-VC1_all_in_one/F01/words/01/03/
cropped/F01/words/01/03/color_001.jpg
cropped/F01/words/01/03/color_002.jpg
cropped/F01/words/01/03/color_003.jpg
cropped/F01/words/01/03/color_004.jpg
cropped/F01/words/01/03/color_005.jpg
cropped/F01/words/01/03/color_006.jpg
cropped/F01/words/01

In [8]:
os.listdir('D:/Lipreading/cropped')

['F01',
 'F02',
 'F04',
 'F05',
 'F06',
 'F07',
 'F08',
 'F09',
 'F10',
 'F11',
 'M01',
 'M02',
 'M04',
 'M07',
 'M08']

In [9]:
max_seq_length = 22

X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []


MAX_WIDTH = 100
MAX_HEIGHT = 100

In [10]:
from skimage.transform import resize
import time

In [11]:
t1 = time.time()
UNSEEN_VALIDATION_SPLIT = ['F07', 'M02']
UNSEEN_TEST_SPLIT = ['F04','M01']

# people = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
# data_types = ['words']
# folder_enum = ['01','02','03','04','05']
# instances = ['01','02']

directory = "D:\Lipreading\cropped"

for person_id in people:
    tx1 = time.time()
    for data_type in data_types:
        for word_index, word in enumerate(folder_enum):
#             print(f"Word : '{words[word_index]}'")
            for iteration in instances:
                path = os.path.join(directory, person_id, data_type, word, iteration)
                filelist = sorted(os.listdir(path + '/'))
                sequence = []
                for img_name in filelist:
                    if img_name.startswith('color'):
                        image = imageio.imread(path + '/' + img_name)
                        image = resize(image, (MAX_WIDTH, MAX_HEIGHT))
                        image = 255 * image
                        # Convert to integer data type pixels.
                        image = image.astype(np.uint8)
                        sequence.append(image)
                pad_array = [np.zeros((MAX_WIDTH, MAX_HEIGHT))]
                sequence.extend(pad_array * (max_seq_length - len(sequence)))
                #sequence = np.array(sequence)

                if person_id in UNSEEN_TEST_SPLIT:
                    X_test.append(sequence)
                    y_test.append(word_index)
                elif person_id in UNSEEN_VALIDATION_SPLIT:
                    X_val.append(sequence)
                    y_val.append(word_index)
                else:
                    X_train.append(sequence)
                    y_train.append(word_index)
    tx2 = time.time()
    print(f'Finished reading images for person {person_id}. Time taken : {tx2 - tx1} secs.')

t2 = time.time()
print(f"Time taken for creating constant size 3D Tensors from those cropped lip regions : {t2 - t1} secs.")




C:\Users\satvi\AppData\Local\Temp\ipykernel_24316\3475534889.py:23: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(path + '/' + img_name)


Finished reading images for person F01. Time taken : 5.71085786819458 secs.
Finished reading images for person F02. Time taken : 5.924845218658447 secs.
Finished reading images for person F04. Time taken : 8.885935544967651 secs.
Finished reading images for person F05. Time taken : 8.462132930755615 secs.
Finished reading images for person F06. Time taken : 8.077586889266968 secs.
Finished reading images for person F07. Time taken : 6.465874671936035 secs.
Finished reading images for person F08. Time taken : 3.585416316986084 secs.
Finished reading images for person F09. Time taken : 3.049009084701538 secs.
Finished reading images for person F10. Time taken : 2.0326755046844482 secs.
Finished reading images for person F11. Time taken : 2.0385875701904297 secs.
Finished reading images for person M01. Time taken : 2.7816762924194336 secs.
Finished reading images for person M02. Time taken : 3.3472540378570557 secs.
Finished reading images for person M04. Time taken : 2.837090253829956 se

In [12]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

In [13]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

In [14]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(1100, 22, 100, 100)
(200, 22, 100, 100)
(200, 22, 100, 100)


In [15]:
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [16]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(1100,)
(200,)
(200,)


In [17]:
def normalize_it(X):
    v_min = X.min(axis=(2, 3), keepdims=True)
    v_max = X.max(axis=(2, 3), keepdims=True)

    # Calculate the range and replace zero values with epsilon to avoid division by zero
    range_values = np.where((v_max - v_min) == 0, np.finfo(float).eps, v_max - v_min)

    # Normalize the data
    X_normalized = np.where(range_values == 0, 0, (X - v_min) / range_values)

    return X_normalized

In [18]:
from keras.utils import to_categorical


In [19]:
X_train = normalize_it(X_train)
X_val = normalize_it(X_val)
X_test = normalize_it(X_test)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
y_val = to_categorical(y_val, 10)

from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)
X_val, y_val = shuffle(X_val, y_val, random_state=0)

In [20]:
X_train = np.expand_dims(X_train, axis=4)
X_val = np.expand_dims(X_val, axis=4)
X_test = np.expand_dims(X_test, axis=4)

In [21]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(1100, 22, 100, 100, 1)
(200, 22, 100, 100, 1)
(200, 22, 100, 100, 1)


In [22]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(1100, 10)
(200, 10)
(200, 10)


SVM

In [51]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Flatten the 3D tensors for SVM input
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Repeat the labels to match the number of samples in X_train_flat
y_train_flat = np.repeat(y_train, X_train.shape[2] * X_train.shape[3])

# Verify the shapes of input features and target labels
print("X_train_flat shape:", X_train_flat.shape)
print("y_train_flat shape:", y_train_flat.shape)

# Initialize and train the SVM classifier
svm_classifier = SVC(kernel='linear', random_state=0)  # You can change the kernel as needed
svm_classifier.fit(X_train_flat, y_train_flat)

# Make predictions on validation set
y_val_pred = svm_classifier.predict(X_val_flat)

# Calculate accuracy on validation set
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)

# Make predictions on test set
y_test_pred = svm_classifier.predict(X_test_flat)

# Calculate accuracy on test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)


X_train_flat shape: (1100, 220000)
y_train_flat shape: (11000000,)


ValueError: Found input variables with inconsistent numbers of samples: [1100, 11000000]

In [23]:
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Activation, ZeroPadding3D, TimeDistributed, LSTM, GRU, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv3D, MaxPooling3D



In [24]:
model = Sequential()

# 1st layer group
model.add(Conv3D(64, (3, 3, 3), strides = 1, input_shape=(22, 100, 100, 1), activation='relu', padding='valid'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add(Conv3D(128, (3, 3, 3), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add(Conv3D(256, (2, 2, 2), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add(Conv3D(512, (1, 1, 1), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(1, 1, 1), strides=2))

model.add((Flatten()))

# # FC layers group
model.add(Dense(4096, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(.5))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adagrad', metrics=['accuracy'])
model.summary()

print(X_val.shape)
print(y_val.shape)
print(X_train.shape)
print(y_train.shape)



c:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 20, 98, 98, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 10, 49, 49, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 8, 47, 47, 128) │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 4, 23, 23, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 3, 22, 22, 256) │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 1, 11, 11, 256) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (None, 1, 11, 11, 512) │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (None, 1, 6, 6, 512)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    75,501,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2048)           │     8,390,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │        10,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 86,617,738 (330.42 MB)

 Trainable params: 86,617,738 (330.42 MB)

 Non-trainable params: 0 (0.00 B)

(200, 22, 100, 100, 1)
(200, 10)
(1100, 22, 100, 100, 1)
(1100, 10)


In [25]:
t1 = time.time()
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)
t2 = time.time()
print()
print(f"Training time : {t2 - t1} secs.")

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 247s 7s/step - accuracy: 0.1069 - loss: 2.3040 - val_accuracy: 0.1650 - val_loss: 2.3018
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 255s 7s/step - accuracy: 0.1174 - loss: 2.3018 - val_accuracy: 0.1600 - val_loss: 2.3016
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 247s 7s/step - accuracy: 0.0994 - loss: 2.3025 - val_accuracy: 0.1650 - val_loss: 2.3013
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 262s 7s/step - accuracy: 0.0999 - loss: 2.3053 - val_accuracy: 0.1350 - val_loss: 2.3011
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 257s 7s/step - accuracy: 0.1078 - loss: 2.3038 - val_accuracy: 0.1000 - val_loss: 2.3009
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 266s 7s/step - accuracy: 0.1058 - loss: 2.3029 - val_accuracy: 0.1400 - val_loss: 2.3005
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 251s 7s/step - accuracy: 0.0947 - loss: 2.3015 - val_accuracy: 0.1450 - val_loss: 2.3005
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 247s 7s/step - accuracy: 0.1112 - loss: 2.2973 - val_accuracy: 0.1350 - v